In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：
bounds = [(0,3),(0,4)]
x_opt = [2.32952019747762,3.17849307411774]
y_opt = -5.50801327159536
def G24(X):
    y = -X[0]-X[1]
    g1 = -2*X[0]**4+8*X[0]**3-8*X[0]**2+X[1]-2
    g2 = -4*X[0]**4+32*X[0]**3-88*X[0]**2+96*X[0]+X[1]-36
    
    return y,g1,g2

In [4]:
y = lambda X : G24(X)[0]
g1 = lambda X : G24(X)[1]
g2 = lambda X : G24(X)[2]
x0 = [0,0]
# x0 = [3,4]

In [5]:
maxfunction(g1,x0,bounds)

     fun: -1.9999999999999991
     jac: array([ 1.1920929e-07, -1.0000000e+00])
 message: 'Optimization terminated successfully'
    nfev: 9
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0., 4.])

In [6]:
### 适应度函数
def fitnessG24_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG24_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 0
    fmin = -7
    vmax = 4
    e = 0.1
    n = 2
    
    res = G24(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:3]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness     

In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG24_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG24_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [8]:
PSO = PSO_fit(pN=4000, dim=2, max_iter=1000, bounds=bounds)

In [9]:
PSO.init_Population()

In [ ]:
PSO.iterator()

[4.55674174 4.9596865  4.41061717 ... 4.34668663 4.5607924  4.77323418]
[5.14200218 4.9596865  5.147011   ... 5.37210139 4.5607924  4.9320272 ]
[5.14200218 5.13273147 5.147011   ... 5.37210139 4.5607924  5.02428593]
[5.14200218 5.20649739 5.29433244 ... 5.37210139 5.34617034 5.36265047]
[5.14200218 5.20649739 5.29433244 ... 5.37210139 5.38270171 5.36265047]
[5.35478616 5.20649739 5.29433244 ... 5.37210139 5.38270171 5.36265047]
[5.35478616 5.20649739 5.29433244 ... 5.37210139 5.38270171 5.36265047]
[5.35478616 5.32599668 5.36469748 ... 5.40180366 5.38270171 5.36265047]
[5.35478616 5.32599668 5.36858307 ... 5.40180366 5.38270171 5.36265047]
[5.41038737 5.32599668 5.36858307 ... 5.43059046 5.4684726  5.36265047]
[5.41155626 5.32599668 5.42003394 ... 5.43059046 5.4684726  5.36265047]
[5.41155626 5.36579028 5.42003394 ... 5.43059046 5.4684726  5.37126872]
[5.41155626 5.36579028 5.42003394 ... 5.43059046 5.4684726  5.45591615]
[5.41155626 5.41155626 5.4684726  ... 5.44556566 5.4684726  5.45

[5.5079778  5.50800075 5.50797881 ... 5.50797881 5.50799073 5.50797384]
[5.5079778  5.50800075 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.5079778  5.50800075 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.5079778  5.50800545 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.5079778  5.50800545 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.5079778  5.50800545 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.50800281 5.50800545 5.50798462 ... 5.50797881 5.50799073 5.50797384]
[5.50800281 5.50800545 5.50798462 ... 5.50798095 5.50799073 5.50797384]
[5.50800281 5.50800545 5.50798462 ... 5.50798095 5.50800281 5.50797384]
[5.50800281 5.50800545 5.50800545 ... 5.50798095 5.50800281 5.50797384]
[5.50800281 5.50800545 5.50800545 ... 5.50798095 5.50800281 5.50797384]
[5.50800281 5.50800545 5.50800545 ... 5.50798095 5.50800281 5.50800545]
[5.50800281 5.50800545 5.50800545 ... 5.50798095 5.50800281 5.50800545]
[5.50800281 5.50800545 5.50800545 ... 5.50798095 5.50800281 5.50

[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50

[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50801327]
[5.50801327 5.50801327 5.50801327 ... 5.50801327 5.50801327 5.50